In [1]:
# Loading raw text dataset

!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-10-14 19:00:31--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  1.69MB/s    in 0.6s    

2025-10-14 19:00:32 (1.69 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
with open("input.txt","r") as f:
    data = f.read()

In [4]:
print("total characters in the data:",len(data))
print("total unique characters in the data:",len(list(set(data))))

total characters in the data: 1115394
total unique characters in the data: 65


In [5]:
print("Tiny sample of the data")
print(data[:500])

Tiny sample of the data
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [7]:
## Store unique char and vocab size
unique_char = sorted(list(set(data)))
vocab_size = len(unique_char)
print("Unique characters in the data")
print(''.join(unique_char))

Unique characters in the data

 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [9]:
## simple text to token generation
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(unique_char) }
itos = { i:ch for i,ch in enumerate(unique_char) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

ecd = encode("gpt from scratch")
print(ecd)
print(decode(ecd))

[45, 54, 58, 1, 44, 56, 53, 51, 1, 57, 41, 56, 39, 58, 41, 46]
gpt from scratch


In [12]:
# Encode the whole data
import torch
encoded_data = torch.tensor(encode(data), dtype = torch.long)
print("Encoded data:\n",encoded_data[:100])

Encoded data:
 tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [13]:
# Train test split
n = int(0.9*len(encoded_data))
train_data = encoded_data[:n]
val_data = encoded_data[n:]

In [14]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
# inputs and target
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is {target}")

when input is tensor([18]) the target is 47
when input is tensor([18, 47]) the target is 56
when input is tensor([18, 47, 56]) the target is 57
when input is tensor([18, 47, 56, 57]) the target is 58
when input is tensor([18, 47, 56, 57, 58]) the target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is 58


In [17]:
torch.manual_seed(21)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target is {target}")

inputs:
torch.Size([4, 8])
tensor([[43, 39, 56, 58, 57,  2,  0,  0],
        [ 1, 45, 53, 53, 42,  1, 58, 53],
        [44,  1, 39, 52, 53, 58, 46, 43],
        [58, 61, 47, 52, 52,  5, 42,  1]])
targets:
torch.Size([4, 8])
tensor([[39, 56, 58, 57,  2,  0,  0, 32],
        [45, 53, 53, 42,  1, 58, 53,  1],
        [ 1, 39, 52, 53, 58, 46, 43, 56],
        [61, 47, 52, 52,  5, 42,  1, 50]])
----
when input is [43] the target is 39
when input is [43, 39] the target is 56
when input is [43, 39, 56] the target is 58
when input is [43, 39, 56, 58] the target is 57
when input is [43, 39, 56, 58, 57] the target is 2
when input is [43, 39, 56, 58, 57, 2] the target is 0
when input is [43, 39, 56, 58, 57, 2, 0] the target is 0
when input is [43, 39, 56, 58, 57, 2, 0, 0] the target is 32
when input is [1] the target is 45
when input is [1, 45] the target is 53
when input is [1, 45, 53] the target is 53
when input is [1, 45, 53, 53] the target is 42
when input is [1, 45, 53, 53, 42] the target is

In [18]:
# Simple bigram language model
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.7627, grad_fn=<NllLossBackward0>)

DjbJGMfeLPQXfSh V,u-yFS HMcgxEOHD-EBx
MAWG&$nUfNye.QzWRYOzfyeqQWp?XLk.N,xExh: !
DAW- bRQOFTirDnhzA&s
